In [31]:
import pandas as pd

In [32]:
df_preventiva = pd.read_excel('PREVENTIVA.xlsx')

In [33]:
df_carreta = pd.read_excel('CARRETA.xlsx')

In [5]:
df_Esl = pd.read_excel('ESL.xlsx')

In [6]:

df_mobile = pd.read_excel('MOBILE.xlsx')

In [45]:
df_bipe = pd.read_excel('BIPE.xlsx')

In [8]:
df_bipe_de_notas = pd.read_excel("BIPE DE NOTAS.xlsx")

In [27]:
df_preventiva.head(5)

,cidade_cliente,PEDIDO 1P/FULL
0,GOIANIA,1123564893
1,GOIANIA,1123599834
2,GOIANIA,1123493300
3,GOIANIA,1116744823
4,GOIANIA,1123420028


In [28]:
df_carreta.head(2)

,PEDIDO,NF`s,ROMANEIO/ID,TRANSP / MOTO,CD/FILIAL,DATA_ROMANEIO,DATA ENTRADA,HORA,PREVISÃO ENTREGA,TIPO,PEDIDOS SISTEMA,CHAVE,OBS,UUID
0,1106521713,158581,114206587,7459091,1100,2025-10-31,2025-11-01,01:00:00,2025-11-01,Porta a Porta,NaN,52251047960950110733550070001585811061419842,NaN,c41b6316-569d-44f1-acd2-b5d7be570e63
1,1107281555,159005,114206587,7459091,1100,2025-10-31,2025-11-01,01:00:00,2025-11-01,Porta a Porta,NaN,52251047960950110733550070001590051071263336,NaN,60dd74af-a716-4cd8-b389-407e6a215b83


In [34]:
df_merge = pd.merge(df_preventiva, df_carreta[['PEDIDO','NF`s','CHAVE','PREVISÃO ENTREGA']], left_on='PEDIDO 1P/FULL', right_on='PEDIDO', how='left').drop(columns=['PEDIDO'])

In [35]:
df_merge.head(5)

,cidade_cliente,PEDIDO 1P/FULL,NF`s,CHAVE,PREVISÃO ENTREGA
0,GOIANIA,1123564893,287286.0,52251247960950110733550070002872861081230695,2025-12-18
1,GOIANIA,1123599834,287299.0,52251247960950110733550070002872991011896358,2025-12-18
2,GOIANIA,1123493300,287388.0,52251247960950110733550070002873881040835800,2025-12-18
3,GOIANIA,1116744823,271521.0,52251247960950110733550070002715211070853387,2025-12-12
4,GOIANIA,1123420028,287455.0,52251247960950110733550070002874551085018288,2025-12-18


In [36]:
df_mobile.head(5)

,Pedido Cliente,Pedido,UUID,Romaneio,Transportador,Nota Fiscal,Numero de Série,Data Importação Pedido,Data de Captura,Data de Processamento,Entregador,Tipo
0,1482670513898702,1.106522e+09,c41b6316-569d-44f1-acd2-b5d7be570e63,114206587.0,PORTA A PORTA TRANSPORTES LTDA,158581,7,01/11/25 06:40,01/11/25 17:38,01/11/25 17:38,john Andrew,Entrega Realizada Normalmente
1,1107281555,1.107282e+09,60dd74af-a716-4cd8-b389-407e6a215b83,114206587.0,PORTA A PORTA TRANSPORTES LTDA,159005,7,01/11/25 06:40,01/11/25 17:37,01/11/25 17:37,john Andrew,Entrega Realizada Normalmente
2,1107291288,1.107291e+09,9c736fe9-6171-4b41-ae32-cedcfa0d2900,114206587.0,PORTA A PORTA TRANSPORTES LTDA,159007,7,01/11/25 06:40,01/11/25 17:47,01/11/25 17:48,john Andrew,Entrega Realizada Normalmente
3,1107289924,1.107290e+09,71b9c443-8e75-4846-87c1-8ae58546f42a,114206587.0,PORTA A PORTA TRANSPORTES LTDA,159006,7,01/11/25 06:40,01/11/25 17:37,01/11/25 17:37,john Andrew,Entrega Realizada Normalmente
4,1107221480,1.107221e+09,cc7e42dd-f7ca-4723-be13-744301fefb02,114206587.0,PORTA A PORTA TRANSPORTES LTDA,158806,7,01/11/25 06:40,01/11/25 17:38,01/11/25 17:38,john Andrew,Entrega Realizada Normalmente


In [38]:
df_merge_2 = df_merge.merge(df_mobile[['Pedido','Tipo','Entregador']], left_on='PEDIDO 1P/FULL', right_on='Pedido', how='left').drop(columns=['Pedido'])

In [40]:
df_merge_2.head(5)

,cidade_cliente,PEDIDO 1P/FULL,NF`s,CHAVE,PREVISÃO ENTREGA,Tipo,Entregador
0,GOIANIA,1123564893,287286.0,52251247960950110733550070002872861081230695,2025-12-18,Saiu para Entrega,ROBERT CARNEIRO DE ANDRADE (GOIANIA)
1,GOIANIA,1123599834,287299.0,52251247960950110733550070002872991011896358,2025-12-18,Saiu para Entrega,TULIO CESAR DE OLIVEIRA SOUZA
2,GOIANIA,1123493300,287388.0,52251247960950110733550070002873881040835800,2025-12-18,Em rota de entrega,Anderson soares pinto
3,GOIANIA,1116744823,271521.0,52251247960950110733550070002715211070853387,2025-12-12,Em rota de entrega,Anderson soares pinto
4,GOIANIA,1123420028,287455.0,52251247960950110733550070002874551085018288,2025-12-18,Saiu para Entrega,ROBERT CARNEIRO DE ANDRADE (GOIANIA)


In [41]:
df_merge_3 = pd.merge(df_merge_2,df_Esl[[ 'Nota Fiscal/Chave NF-e','Última ocorrência/Observações', 'Pessoa/Nome', 'Última ocorrência/Data ocorrência']], left_on='CHAVE', right_on= 'Nota Fiscal/Chave NF-e', how='left').drop(columns=['Nota Fiscal/Chave NF-e'])

In [43]:
df_merge_3.head(2)

,cidade_cliente,PEDIDO 1P/FULL,NF`s,CHAVE,PREVISÃO ENTREGA,Tipo,Entregador,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência
0,GOIANIA,1123564893,287286.0,52251247960950110733550070002872861081230695,2025-12-18,Saiu para Entrega,ROBERT CARNEIRO DE ANDRADE (GOIANIA),Manifesto 55375 - DOMICILIAR TRANSPORTES EIRELI,ROBERT CARNEIRO DE ANDRADE (GOIANIA),2025-12-18 08:16:00
1,GOIANIA,1123599834,287299.0,52251247960950110733550070002872991011896358,2025-12-18,Saiu para Entrega,TULIO CESAR DE OLIVEIRA SOUZA,Manifesto 55371 - DOMICILIAR TRANSPORTES EIRELI,TULIO CESAR DE OLIVEIRA SOUZA (GOIANIA),2025-12-18 07:54:00


In [46]:
df_bipe.head(2)

,PEDIDO,INFO
0,1024536543,BIPE
1,1067558615,BIPE


In [48]:
df_merge_4 = pd.merge(df_merge_3, df_bipe[['PEDIDO','INFO']], left_on='PEDIDO 1P/FULL', right_on='PEDIDO', how='left').drop(columns=['PEDIDO'])

In [50]:
df_merge_4.head(2)

,cidade_cliente,PEDIDO 1P/FULL,NF`s,CHAVE,PREVISÃO ENTREGA,Tipo,Entregador,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,INFO
0,GOIANIA,1123564893,287286.0,52251247960950110733550070002872861081230695,2025-12-18,Saiu para Entrega,ROBERT CARNEIRO DE ANDRADE (GOIANIA),Manifesto 55375 - DOMICILIAR TRANSPORTES EIRELI,ROBERT CARNEIRO DE ANDRADE (GOIANIA),2025-12-18 08:16:00,NaN
1,GOIANIA,1123599834,287299.0,52251247960950110733550070002872991011896358,2025-12-18,Saiu para Entrega,TULIO CESAR DE OLIVEIRA SOUZA,Manifesto 55371 - DOMICILIAR TRANSPORTES EIRELI,TULIO CESAR DE OLIVEIRA SOUZA (GOIANIA),2025-12-18 07:54:00,NaN


In [51]:
df_bipe_de_notas.head(2)

,NF,CHAVE,OCORRENCIA,CD
0,238484,52251147960950110733550070002384841014816100,PENDENCIA,1100
1,285586,52251247960950110733550070002855861052381041,PENDENCIA,1100


In [53]:
df_merge_5 = pd.merge(df_merge_4, df_bipe_de_notas[['NF', 'OCORRENCIA ']], left_on='NF`s', right_on='NF', how='left').drop(columns=['NF'])

In [54]:
df_merge_5.head(2)

,cidade_cliente,PEDIDO 1P/FULL,NF`s,CHAVE,PREVISÃO ENTREGA,Tipo,Entregador,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,INFO,OCORRENCIA
0,GOIANIA,1123564893,287286.0,52251247960950110733550070002872861081230695,2025-12-18,Saiu para Entrega,ROBERT CARNEIRO DE ANDRADE (GOIANIA),Manifesto 55375 - DOMICILIAR TRANSPORTES EIRELI,ROBERT CARNEIRO DE ANDRADE (GOIANIA),2025-12-18 08:16:00,NaN,NaN
1,GOIANIA,1123599834,287299.0,52251247960950110733550070002872991011896358,2025-12-18,Saiu para Entrega,TULIO CESAR DE OLIVEIRA SOUZA,Manifesto 55371 - DOMICILIAR TRANSPORTES EIRELI,TULIO CESAR DE OLIVEIRA SOUZA (GOIANIA),2025-12-18 07:54:00,NaN,NaN


In [55]:
df_merge_5.to_excel('29-12-2025.xlsx', sheet_name='Relatorio_diario', index=False)